Вот пример приложения на `wxPython`, которое читает данные из `Excel` с помощью `pandas` и отображает их в таблице `wx.grid.Grid`.

Чтобы использовать этот код, убедитесь, что у вас установлены необходимые библиотеки:
```bash
pip install wxPython pandas openpyxl
```

In [ ]:
import wx
import wx.grid
import pandas as pd
from pathlib import Path
import openpyxl

class ExcelViewerFrame(wx.Frame):
    def __init__(self, parent):
        super().__init__(parent=parent, title='Просмотр Excel', size=(800, 600))

        self.current_file = None  # Путь к текущему файлу
        self.current_sheet = None  # Имя текущего листа
        self.sheets = []  # Список листов в файле

        # Создаем главную панель
        self.panel = wx.Panel(self)

        # Создаем вертикальный сайзер для компоновки элементов
        main_sizer = wx.BoxSizer(wx.VERTICAL)

        # Создаем горизонтальный сайзер для кнопок и выпадающего списка
        button_sizer = wx.BoxSizer(wx.HORIZONTAL)

        # Создаем кнопку для выбора файла
        self.load_button = wx.Button(self.panel, label='Открыть Excel файл')
        self.load_button.Bind(wx.EVT_BUTTON, self.on_load_file)
        button_sizer.Add(self.load_button, 0, wx.ALL, 5)

        # Создаем выпадающий список для выбора листа
        self.sheet_choice = wx.Choice(self.panel, choices=[])
        self.sheet_choice.Bind(wx.EVT_CHOICE, self.on_sheet_selected)
        self.sheet_choice.Disable()  # Изначально отключен
        button_sizer.Add(self.sheet_choice, 0, wx.ALL | wx.EXPAND, 5)

        # Создаем кнопку сохранения
        self.save_button = wx.Button(self.panel, label='Сохранить изменения')
        self.save_button.Bind(wx.EVT_BUTTON, self.on_save)
        self.save_button.Disable()  # Изначально отключена
        button_sizer.Add(self.save_button, 0, wx.ALL, 5)

        # Добавляем сайзер с кнопками в главный сайзер
        main_sizer.Add(button_sizer, 0, wx.EXPAND)

        # Создаем грид
        self.grid = wx.grid.Grid(self.panel)
        self.grid.CreateGrid(0, 0)  # Изначально создаем пустую таблицу

        # Добавляем грид в главный сайзер
        main_sizer.Add(self.grid, 1, wx.EXPAND | wx.ALL, 5)

        # Устанавливаем сайзер для панели
        self.panel.SetSizer(main_sizer)

        # Создаем строку состояния
        self.statusbar = self.CreateStatusBar()
        self.statusbar.SetStatusText('Готов к работе')

        # Привязываем обработчик правого клика
        self.grid.Bind(wx.grid.EVT_GRID_CELL_RIGHT_CLICK, self.on_cell_right_click)

        # Центрируем окно
        self.Centre()

    def on_load_file(self, event):
        """Обработчик нажатия кнопки загрузки файла"""
        with wx.FileDialog(self, "Выберите Excel файл",
                         wildcard="Excel files (*.xlsx;*.xlsm)|*.xlsx;*.xlsm",
                         style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:

            if fileDialog.ShowModal() == wx.ID_CANCEL:
                return

            pathname = fileDialog.GetPath()

            try:
                # Загружаем список листов
                workbook = openpyxl.load_workbook(pathname, read_only=True, keep_vba=True)
                self.sheets = workbook.sheetnames
                workbook.close()

                # Сохраняем путь к файлу
                self.current_file = pathname

                # Обновляем выпадающий список листов
                self.sheet_choice.SetItems(self.sheets)
                self.sheet_choice.SetSelection(0)
                self.sheet_choice.Enable()

                # Загружаем первый лист
                self.current_sheet = self.sheets[0]
                self.load_excel_sheet(pathname, self.current_sheet)

                # Активируем кнопку сохранения
                self.save_button.Enable()

            except Exception as e:
                wx.MessageBox(f'Ошибка при загрузке файла: {str(e)}', 'Ошибка',
                            wx.OK | wx.ICON_ERROR)

    def on_sheet_selected(self, event):
        """Обработчик выбора листа"""
        selected_sheet = self.sheet_choice.GetString(self.sheet_choice.GetSelection())
        self.current_sheet = selected_sheet
        self.load_excel_sheet(self.current_file, selected_sheet)

    def load_excel_sheet(self, filepath, sheet_name):
        """Загружает данные из конкретного листа Excel файла"""
        # Читаем Excel файл
        df = pd.read_excel(filepath, sheet_name=sheet_name)

        # Очищаем текущий грид
        if self.grid.GetNumberRows() > 0:
            self.grid.DeleteRows(0, self.grid.GetNumberRows())
        if self.grid.GetNumberCols() > 0:
            self.grid.DeleteCols(0, self.grid.GetNumberCols())

        # Создаем новый грид с нужным количеством строк и столбцов
        self.grid.CreateGrid(len(df), len(df.columns))

        # Устанавливаем заголовки столбцов
        for col_idx, col_name in enumerate(df.columns):
            self.grid.SetColLabelValue(col_idx, str(col_name))

        # Заполняем данные
        for row_idx in range(len(df)):
            for col_idx in range(len(df.columns)):
                value = df.iloc[row_idx, col_idx]
                # Преобразуем значение в строку и обрабатываем NaN
                if pd.isna(value):
                    cell_value = ''
                else:
                    cell_value = str(value)
                self.grid.SetCellValue(row_idx, col_idx, cell_value)

        # Автоматически устанавливаем размеры столбцов
        self.grid.AutoSizeColumns()

        # Обновляем статусбар
        filename = Path(filepath).name
        self.statusbar.SetStatusText(f'Загружен файл: {filename}, лист: {sheet_name}')

    def on_save(self, event):
        """Обработчик сохранения изменений"""
        try:
            # Создаем DataFrame из данных в гриде
            data = []
            for row in range(self.grid.GetNumberRows()):
                row_data = []
                for col in range(self.grid.GetNumberCols()):
                    row_data.append(self.grid.GetCellValue(row, col))
                data.append(row_data)

            # Получаем заголовки столбцов
            columns = [self.grid.GetColLabelValue(col) for col in range(self.grid.GetNumberCols())]

            # Создаем DataFrame
            df = pd.DataFrame(data, columns=columns)

            # Загружаем существующий файл
            workbook = openpyxl.load_workbook(self.current_file, keep_vba=True)

            # Создаем writer с режимом, сохраняющим макросы
            with pd.ExcelWriter(self.current_file, engine='openpyxl') as writer:
                writer.book = workbook
                writer.sheets = {ws.title: ws for ws in workbook.worksheets}

                # Записываем DataFrame в конкретный лист
                df.to_excel(writer, sheet_name=self.current_sheet, index=False)

                # Сохраняем изменения
                writer.save()

            self.statusbar.SetStatusText(f'Изменения сохранены в лист: {self.current_sheet}')

        except Exception as e:
            wx.MessageBox(f'Ошибка при сохранении: {str(e)}', 'Ошибка',
                        wx.OK | wx.ICON_ERROR)

    def on_cell_right_click(self, event):
        """Обработчик правого клика по ячейке"""
        row = event.GetRow()
        col = event.GetCol()

        menu = wx.Menu()
        copy_item = menu.Append(wx.ID_ANY, "Копировать")
        self.Bind(wx.EVT_MENU, lambda evt, rc=(row,col): self.on_copy_cell(evt, rc), copy_item)

        self.grid.PopupMenu(menu)
        menu.Destroy()

    def on_copy_cell(self, event, row_col):
        """Копирует содержимое ячейки в буфер обмена"""
        row, col = row_col
        cell_value = self.grid.GetCellValue(row, col)

        if wx.TheClipboard.Open():
            wx.TheClipboard.SetData(wx.TextDataObject(cell_value))
            wx.TheClipboard.Close()
            self.statusbar.SetStatusText(f'Скопировано: {cell_value}')

def main():
    app = wx.App()
    frame = ExcelViewerFrame(None)
    frame.Show()
    app.MainLoop()

if __name__ == '__main__':
    main()

Этот пример включает следующие возможности:

1. **Выбор файла**:
   - Кнопка для открытия диалога выбора Excel-файла
   - Поддержка только файлов .xlsx
   - Обработка ошибок при загрузке файла

2. **Отображение данных**:
   - Автоматическое создание таблицы нужного размера
   - Отображение заголовков столбцов из Excel
   - Корректная обработка пустых значений (NaN)
   - Автоматическая подстройка размеров столбцов

3. **Дополнительные функции**:
   - Строка состояния, показывающая имя загруженного файла
   - Возможность копирования значений ячеек через контекстное меню
   - Очистка предыдущих данных при загрузке нового файла

---

Вы можете расширить этот пример, добавив дополнительные функции:

1. **Фильтрация данных**:

In [ ]:
def add_filter_controls(self):
    filter_sizer = wx.BoxSizer(wx.HORIZONTAL)
    self.filter_text = wx.TextCtrl(self.panel)
    filter_button = wx.Button(self.panel, label='Фильтр')
    filter_button.Bind(wx.EVT_BUTTON, self.on_filter)
    filter_sizer.Add(self.filter_text, 1, wx.ALL, 5)
    filter_sizer.Add(filter_button, 0, wx.ALL, 5)
    return filter_sizer

def on_filter(self, event):
    filter_text = self.filter_text.GetValue().lower()
    for row in range(self.grid.GetNumberRows()):
        match = False
        for col in range(self.grid.GetNumberCols()):
            if filter_text in self.grid.GetCellValue(row, col).lower():
                match = True
                break
        self.grid.ShowRow(row, match)

2. **Сохранение изменений**:

In [ ]:
def add_save_button(self):
    save_button = wx.Button(self.panel, label='Сохранить изменения')
    save_button.Bind(wx.EVT_BUTTON, self.on_save)
    return save_button

def on_save(self, event):
    # Создаем новый DataFrame из данных в гриде
    data = []
    for row in range(self.grid.GetNumberRows()):
        row_data = []
        for col in range(self.grid.GetNumberCols()):
            row_data.append(self.grid.GetCellValue(row, col))
        data.append(row_data)

    df = pd.DataFrame(data, columns=[self.grid.GetColLabelValue(col)
                                   for col in range(self.grid.GetNumberCols())])

    # Сохраняем в Excel
    with wx.FileDialog(self, "Сохранить Excel файл", wildcard="Excel files (*.xlsx)|*.xlsx",
                      style=wx.FD_SAVE | wx.FD_OVERWRITE_PROMPT) as fileDialog:

        if fileDialog.ShowModal() == wx.ID_CANCEL:
            return

        pathname = fileDialog.GetPath()
        try:
            df.to_excel(pathname, index=False)
            self.statusbar.SetStatusText(f'Файл сохранен: {pathname}')
        except Exception as e:
            wx.MessageBox(f'Ошибка при сохранении: {str(e)}', 'Ошибка',
                        wx.OK | wx.ICON_ERROR)

3. **Сортировка по столбцам**:

In [ ]:
def enable_column_sorting(self):
    self.grid.Bind(wx.grid.EVT_GRID_COL_SORT, self.on_col_sort)

def on_col_sort(self, event):
    col = event.GetCol()
    # Получаем данные столбца
    data = []
    for row in range(self.grid.GetNumberRows()):
        data.append((self.grid.GetCellValue(row, col), row))

    # Сортируем
    data.sort()

    # Переупорядочиваем строки
    for new_row, (_, old_row) in enumerate(data):
        for col in range(self.grid.GetNumberCols()):
            value = self.grid.GetCellValue(old_row, col)
            self.grid.SetCellValue(new_row, col, value)

Эти дополнительные функции можно интегрировать в основной код по мере необходимости.

---

In [ ]:
# вариант 2 - с дополнительными функциями

import wx
import wx.grid
import pandas as pd
from pathlib import Path
import openpyxl

class ExcelViewerFrame(wx.Frame):
    def __init__(self, parent):
        super().__init__(parent=parent, title='Просмотр Excel', size=(800, 600))

        self.current_file = None
        self.current_sheet = None
        self.sheets = []
        self.original_data = None  # Сохраняем оригинальные данные для фильтрации

        # Создаем главную панель
        self.panel = wx.Panel(self)

        # Создаем вертикальный сайзер для компоновки элементов
        main_sizer = wx.BoxSizer(wx.VERTICAL)

        # Создаем горизонтальный сайзер для верхних элементов управления
        top_sizer = wx.BoxSizer(wx.HORIZONTAL)

        # Создаем кнопку для выбора файла
        self.load_button = wx.Button(self.panel, label='Открыть Excel файл')
        self.load_button.Bind(wx.EVT_BUTTON, self.on_load_file)
        top_sizer.Add(self.load_button, 0, wx.ALL, 5)

        # Создаем выпадающий список для выбора листа
        self.sheet_choice = wx.Choice(self.panel, choices=[])
        self.sheet_choice.Bind(wx.EVT_CHOICE, self.on_sheet_selected)
        self.sheet_choice.Disable()
        top_sizer.Add(self.sheet_choice, 0, wx.ALL | wx.EXPAND, 5)

        # Создаем кнопку сохранения
        self.save_button = wx.Button(self.panel, label='Сохранить изменения')
        self.save_button.Bind(wx.EVT_BUTTON, self.on_save)
        self.save_button.Disable()
        top_sizer.Add(self.save_button, 0, wx.ALL, 5)

        main_sizer.Add(top_sizer, 0, wx.EXPAND)

        # Создаем сайзер для фильтрации
        filter_sizer = wx.BoxSizer(wx.HORIZONTAL)

        # Добавляем метку для поля фильтрации
        filter_label = wx.StaticText(self.panel, label="Фильтр:")
        filter_sizer.Add(filter_label, 0, wx.ALL | wx.CENTER, 5)

        # Создаем поле для фильтрации
        self.filter_text = wx.TextCtrl(self.panel)
        self.filter_text.Bind(wx.EVT_TEXT, self.on_filter)
        filter_sizer.Add(self.filter_text, 1, wx.ALL | wx.EXPAND, 5)

        # Добавляем кнопку сброса фильтра
        self.clear_filter_button = wx.Button(self.panel, label='Сбросить фильтр')
        self.clear_filter_button.Bind(wx.EVT_BUTTON, self.on_clear_filter)
        filter_sizer.Add(self.clear_filter_button, 0, wx.ALL, 5)

        main_sizer.Add(filter_sizer, 0, wx.EXPAND)

        # Создаем грид
        self.grid = wx.grid.Grid(self.panel)
        self.grid.CreateGrid(0, 0)

        # Включаем сортировку по клику на заголовок
        self.grid.EnableDragColMove(False)
        self.grid.EnableDragGridSize(False)
        self.grid.UseNativeColHeader(True)
        self.grid.Bind(wx.grid.EVT_GRID_COL_SORT, self.on_col_sort)

        main_sizer.Add(self.grid, 1, wx.EXPAND | wx.ALL, 5)

        # Устанавливаем сайзер для панели
        self.panel.SetSizer(main_sizer)

        # Создаем строку состояния
        self.statusbar = self.CreateStatusBar()
        self.statusbar.SetStatusText('Готов к работе')

        # Привязываем обработчик правого клика
        self.grid.Bind(wx.grid.EVT_GRID_CELL_RIGHT_CLICK, self.on_cell_right_click)

        # Центрируем окно
        self.Centre()

    def on_load_file(self, event):
        with wx.FileDialog(self, "Выберите Excel файл",
                         wildcard="Excel files (*.xlsx;*.xlsm)|*.xlsx;*.xlsm",
                         style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:

            if fileDialog.ShowModal() == wx.ID_CANCEL:
                return

            pathname = fileDialog.GetPath()

            try:
                # Загружаем список листов
                workbook = openpyxl.load_workbook(pathname, read_only=True, keep_vba=True)
                self.sheets = workbook.sheetnames
                workbook.close()

                self.current_file = pathname
                self.sheet_choice.SetItems(self.sheets)
                self.sheet_choice.SetSelection(0)
                self.sheet_choice.Enable()
                self.current_sheet = self.sheets[0]
                self.load_excel_sheet(pathname, self.current_sheet)
                self.save_button.Enable()

            except Exception as e:
                wx.MessageBox(f'Ошибка при загрузке файла: {str(e)}', 'Ошибка',
                            wx.OK | wx.ICON_ERROR)

    # def load_excel_sheet(self, filepath, sheet_name):
    #     """Загружает данные из конкретного листа Excel файла"""
    #     # Читаем Excel файл
    #     self.original_data = pd.read_excel(filepath, sheet_name=sheet_name)
    #     self.display_data(self.original_data)

    def load_excel_sheet(self, filepath, sheet_name):
        """Загружает данные из конкретного листа Excel файла"""
        try:
            # Показываем диалог прогресса
            dlg = wx.ProgressDialog("Загрузка данных",
                                "Пожалуйста, подождите...",
                                maximum=100,
                                parent=self,
                                style=wx.PD_APP_MODAL | wx.PD_AUTO_HIDE)

            try:
                # Обновляем прогресс
                dlg.Update(30, "Чтение файла...")
                self.original_data = pd.read_excel(filepath, sheet_name=sheet_name)

                dlg.Update(60, "Обработка данных...")
                self.display_data(self.original_data)

                dlg.Update(100, "Завершение...")

                # Обновляем статусбар
                filename = Path(filepath).name
                self.statusbar.SetStatusText(f'Загружен файл: {filename}, лист: {sheet_name}')

            finally:
                dlg.Destroy()

        except Exception as e:
            wx.MessageBox(f'Ошибка при загрузке листа: {str(e)}', 'Ошибка',
                        wx.OK | wx.ICON_ERROR)

    def display_data(self, df):
        """Отображает DataFrame в гриде"""
        # Очищаем текущий грид
        if self.grid.GetNumberRows() > 0:
            self.grid.DeleteRows(0, self.grid.GetNumberRows())
        if self.grid.GetNumberCols() > 0:
            self.grid.DeleteCols(0, self.grid.GetNumberCols())

        # Создаем новый грид
        self.grid.CreateGrid(len(df), len(df.columns))

        # Устанавливаем заголовки столбцов
        for col_idx, col_name in enumerate(df.columns):
            self.grid.SetColLabelValue(col_idx, str(col_name))

        # Заполняем данные
        for row_idx in range(len(df)):
            for col_idx in range(len(df.columns)):
                value = df.iloc[row_idx, col_idx]
                cell_value = '' if pd.isna(value) else str(value)
                self.grid.SetCellValue(row_idx, col_idx, cell_value)

        # Автоматически устанавливаем размеры столбцов
        self.grid.AutoSizeColumns()

    def on_filter(self, event):
        """Обработчик фильтрации данных"""
        if self.original_data is None:
            return

        filter_text = self.filter_text.GetValue().lower()

        if not filter_text:
            self.display_data(self.original_data)
            return

        # Фильтруем данные
        mask = self.original_data.astype(str).apply(lambda x: x.str.lower()).apply(
            lambda x: x.str.contains(filter_text, na=False)).any(axis=1)
        filtered_data = self.original_data[mask]

        # Отображаем отфильтрованные данные
        self.display_data(filtered_data)

        # Обновляем статус
        self.statusbar.SetStatusText(f'Найдено записей: {len(filtered_data)}')

    def on_clear_filter(self, event):
        """Обработчик сброса фильтра"""
        self.filter_text.SetValue('')
        if self.original_data is not None:
            self.display_data(self.original_data)
            self.statusbar.SetStatusText('Фильтр сброшен')

    def on_col_sort(self, event):
        """Обработчик сортировки по столбцу"""
        col = event.GetCol()
        col_name = self.grid.GetColLabelValue(col)

        if self.original_data is None:
            return

        # Получаем текущие отображаемые данные
        current_data = self.get_current_grid_data()

        # Определяем порядок сортировки (переключаем при каждом клике)
        ascending = not hasattr(self, 'last_sort_ascending') or not self.last_sort_ascending
        self.last_sort_ascending = ascending

        # Сортируем данные
        sorted_data = current_data.sort_values(by=col_name, ascending=ascending)

        # Отображаем отсортированные данные
        self.display_data(sorted_data)

        # Обновляем статус
        sort_direction = "по возрастанию" if ascending else "по убыванию"
        self.statusbar.SetStatusText(f'Отсортировано по {col_name} {sort_direction}')

    def get_current_grid_data(self):
        """Получает текущие данные из грида в виде DataFrame"""
        data = []
        for row in range(self.grid.GetNumberRows()):
            row_data = []
            for col in range(self.grid.GetNumberCols()):
                row_data.append(self.grid.GetCellValue(row, col))
            data.append(row_data)

        columns = [self.grid.GetColLabelValue(col) for col in range(self.grid.GetNumberCols())]
        return pd.DataFrame(data, columns=columns)

    def on_sheet_selected(self, event):
        selected_sheet = self.sheet_choice.GetString(self.sheet_choice.GetSelection())
        self.current_sheet = selected_sheet
        self.load_excel_sheet(self.current_file, selected_sheet)

    # def on_save(self, event):
    #     try:
    #         # Получаем текущие данные из грида
    #         df = self.get_current_grid_data()

    #         # Загружаем существующий файл
    #         workbook = openpyxl.load_workbook(self.current_file, keep_vba=True)

    #         # Создаем writer с режимом, сохраняющим макросы
    #         with pd.ExcelWriter(self.current_file, engine='openpyxl') as writer:
    #             writer.book = workbook
    #             writer.sheets = {ws.title: ws for ws in workbook.worksheets}

    #             # Записываем DataFrame в конкретный лист
    #             df.to_excel(writer, sheet_name=self.current_sheet, index=False)

    #             # Сохраняем изменения
    #             writer.save()

    #         self.statusbar.SetStatusText(f'Изменения сохранены в лист: {self.current_sheet}')

    #     except Exception as e:
    #         wx.MessageBox(f'Ошибка при сохранении: {str(e)}', 'Ошибка',
    #                     wx.OK | wx.ICON_ERROR)

    def on_save(self, event):
        try:
            # Запрашиваем подтверждение перед сохранением
            dlg = wx.MessageDialog(None,
                                "Вы уверены, что хотите сохранить изменения?",
                                "Подтверждение сохранения",
                                wx.YES_NO | wx.NO_DEFAULT | wx.ICON_QUESTION)

            if dlg.ShowModal() == wx.ID_NO:
                dlg.Destroy()
                return

            dlg.Destroy()

            # Получаем текущие данные из грида
            df = self.get_current_grid_data()

            # Загружаем существующий файл
            workbook = openpyxl.load_workbook(self.current_file, keep_vba=True)

            # Получаем лист, в который будем сохранять данные
            worksheet = workbook[self.current_sheet]

            # Очищаем лист перед записью новых данных
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.value = None

            # Записываем заголовки
            for col_idx, column_name in enumerate(df.columns, 1):
                worksheet.cell(row=1, column=col_idx, value=column_name)

            # Записываем данные
            for row_idx, row_data in enumerate(df.values, 2):
                for col_idx, value in enumerate(row_data, 1):
                    worksheet.cell(row=row_idx, column=col_idx, value=value)

            # Сохраняем файл
            workbook.save(self.current_file)

            self.statusbar.SetStatusText(f'Изменения сохранены в лист: {self.current_sheet}')

            # Показываем сообщение об успешном сохранении
            wx.MessageBox('Файл успешно сохранен!', 'Информация',
                        wx.OK | wx.ICON_INFORMATION)

        except Exception as e:
            wx.MessageBox(f'Ошибка при сохранении: {str(e)}', 'Ошибка',
                        wx.OK | wx.ICON_ERROR)

    def on_cell_right_click(self, event):
        row = event.GetRow()
        col = event.GetCol()

        menu = wx.Menu()
        copy_item = menu.Append(wx.ID_ANY, "Копировать")
        self.Bind(wx.EVT_MENU, lambda evt, rc=(row,col): self.on_copy_cell(evt, rc), copy_item)

        self.grid.PopupMenu(menu)
        menu.Destroy()

    def on_copy_cell(self, event, row_col):
        row, col = row_col
        cell_value = self.grid.GetCellValue(row, col)

        if wx.TheClipboard.Open():
            wx.TheClipboard.SetData(wx.TextDataObject(cell_value))
            wx.TheClipboard.Close()
            self.statusbar.SetStatusText(f'Скопировано: {cell_value}')

def main():
    app = wx.App()
    frame = ExcelViewerFrame(None)
    frame.Show()
    app.MainLoop()

if __name__ == '__main__':
    main()